In [65]:
import pandas as pd
import paramiko
import os
import re
import json


In [66]:
# FTP login information
host = "data.valg.dk"
port = 22
username = "Valg"
password = "Valg"

# Forbindelse til FTP-server
transport = paramiko.Transport((host, port))
transport.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [76]:
sftp.listdir('data/kommunalvalg-134-18-11-2025/')

['geografi',
 'public-key.pem',
 'verifikation',
 'Snitfladebeskrivelser',
 'kandidat-data']

In [68]:
with sftp.open('data/kommunalvalg-134-18-11-2025/geografi/Afstemningsomraade-291020251900.json') as f:
    data = json.load(f)

In [69]:
afstemningssteder = []

for d in data:

    afstemningssteder.append({
        'dagi_id': d['Dagi_id'],
        'navn': d['Navn'],
        'nummer': d['Nummer'],
        'afstemningssted_navn': d['Afstemningssted']['Navn'],
        'kommune_id': d['Kommunekode'],
        'opstillingskreds_nummer': d['Opstillingskredsnummer'],
        'opstillingskreds_dagi_id': d['Opstillingskreds_Dagi_id'],
        'afstemningssted_adresse': d['Afstemningssted']['Adgangsadresse']['Adressebetegnelse']
    })

df_afstemningssteder = pd.DataFrame(afstemningssteder)
df_afstemningssteder      

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk"
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde"
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde"
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev"
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg"
...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde"
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod"
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand"
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk"


In [79]:
# load in the local kommune mapping file in json format
with open('data/kommuner.json') as f:
    kommune_info = json.load(f)

In [81]:
# join the kommune names to the afstemningssteder dataframe
df_afstemningssteder['kommune_navn'] = df_afstemningssteder['kommune_id'].astype(str).map(
    {k['kommune_id']: k['kommune_navn'] for k in kommune_info}
)

# map the kommune dagi ids
df_afstemningssteder['kommune_dagi_id'] = df_afstemningssteder['kommune_id'].astype(str).map(
    {k['kommune_id']: k['kommune_dagi_id'] for k in kommune_info}
)

# round the dagi id to integer and skip any errors
df_afstemningssteder['kommune_dagi_id'] = pd.to_numeric(df_afstemningssteder['kommune_dagi_id'], errors='coerce').round().astype('Int64')

In [82]:
df_afstemningssteder

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse,kommune_navn,kommune_dagi_id
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk",Nyborg,389160
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde",Kerteminde,389159
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde",Kerteminde,389159
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev",Nyborg,389160
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg",Nyborg,389160
...,...,...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde",Varde,389173
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod",Varde,389173
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand",Varde,389173
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk",Varde,389173


In [83]:
# for each kommune dagi id generate a csv file with the afstemningssteder for that kommune
output_dir = 'data/struktureret/kv/valgresultater/afstemningssteder/'

os.makedirs(output_dir, exist_ok=True)

num = 0
for kommune_id, group in df_afstemningssteder.groupby('kommune_id'):
    kommune_navn = group['kommune_navn'].iloc[0].lower().replace(' ', '_')
    output_file = os.path.join(output_dir, f'{kommune_id}_{kommune_navn}_afstemningsområde.csv')
    group['største_parti'] = ''  # placeholder column for største parti
    group.to_csv(output_file, index=False)
    num += 1
print(f'Wrote afstemningssteder for {num} kommuner to {output_dir}')

Wrote afstemningssteder for 98 kommuner to data/struktureret/kv/valgresultater/afstemningssteder/


In [84]:
# also generate a file for each kommune in the folder data/struktureret/kv/valgresultater/kommune/
output_dir_kommune = 'data/struktureret/kv/valgresultater/kommune/'

os.makedirs(output_dir_kommune, exist_ok=True)
# the file should have the columns: kommune_id, kommune_dagi_id, kommune_navn, parti, procent_25, stemmer_25, procent_21, stemmer_21

for kommune_id, group in df_afstemningssteder.groupby('kommune_id'):
    kommune_navn = group['kommune_navn'].iloc[0].lower().replace(' ', '_')
    kommune_dagi_id = group['kommune_dagi_id'].iloc[0]
    output_file = os.path.join(output_dir_kommune, f'{kommune_id}_{kommune_navn}_kommune.csv')
    df_kommune = pd.DataFrame([{
        'kommune_id': kommune_id,
        'kommune_dagi_id': kommune_dagi_id,
        'kommune_navn': group['kommune_navn'].iloc[0],
        'partier': '',
        'procent_25': '',
        'stemmer_25': '',
        'procent_21': '',
        'stemmer_21': ''
    }])
    df_kommune.to_csv(output_file, index=False)